In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np

from promptopt import models
from promptopt import datasets
from promptopt import rlhf
from promptopt import embed

from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:
embedding_model = embed.CLIP()

In [ ]:
net_arch_kwargs = {
  'n_layers': 2,
  'layer_size': 256
}
pref_model = models.PrefModel(embedding_model.n_embedding_dims, net_arch_kwargs=net_arch_kwargs)

In [ ]:
pref_model_train_config = rlhf.get_default_config()
pref_model_train_config.verbose = True

In [ ]:
prompts = [
  'a quick',
  'brown fox',
  'jumped over',
  'the moon'
]

In [ ]:
pref_data = [
  (0, 1, 0),
  (1, 2, 1),
  (1, 3, 1),
  (2, 0, 0),
  (3, 0, 0)
]

In [ ]:
embeddings = embedding_model.embed_strings(prompts)

In [ ]:
embedding_dataset = datasets.EmbeddingDataset(embeddings=list(embeddings))

In [ ]:
pref_dataset = datasets.PrefDataset(embedding_dataset, pref_data=pref_data, train_frac=0.5)

In [ ]:
optimizer = rlhf.RLHF(pref_model, pref_model_train_config, embedding_dataset, pref_dataset)

In [ ]:
candidate_prompts = [
  'quick fox',
  'jumping moon'
]

In [ ]:
candidate_embeddings = embedding_model.embed_strings(prompts)

In [ ]:
candidate_scores = np.array(pref_model.score(candidate_embeddings))

In [ ]:
sorted(list(zip(candidate_prompts, candidate_scores)), key=lambda x: x[1], reverse=True)